I've had a "Testing Python Code" tutorial on the back burner for quite some time. Over and over again I read how it's essential to have tests and particularly high quality tests. Basically I don't need to create a tutorial and I would only have one to provide BU researchers with a live body to pose questions. Ned Batchelder has been presenting [a wonderful tutorial](http://nedbatchelder.com/text/test0.html) about why and how you might test your code, particularly python code. He's also the maintainer of `coverage`, an essential tool to check if you've actually tested every nook and crany. His tutorial is great in video and static form. 

I've dredged up this particular "wouldn't it be great if I presented about this"-tutorial because I've only recently really played with `doctest`. `doctest` is a utility that searches through your docs and doc strings for lines that look like an interactive python session and executes those lines. If the output is consistant, the tests passes. This is the tool for you if you are organized enough to write your docs before you write your tests before you write you code.

How's it work? Well take this example:

In [1]:
%%file phone_how_to.rst

This is the docs for my phone number api. 

    >>> from phones import PhoneNumber

To create a number, you just provide it as a string or integer 

    >>> PhoneNumber(1112223333, 'home')
    [home] (111) 222-3333
    >>> PhoneNumber("1112223333", 'home')
    [home] (111) 222-3333
    
You can also add the optional '1' at the beginning:
    
    >>> PhoneNumber("11112223333", 'home')
    [home] (111) 222-3333
    
Parenthases, dashes or spaces are ignored:    

    >>> number = PhoneNumber("111-222-3333", 'home')
    >>> number
    [home] (111) 222-3333
    
If for any reason you need the raw number, and not the string representation, it's available like so:

    >>> number.raw_number
    '1112223333'

Writing phone_how_to.rst


Now to run doc test on this code, you run the doctest module as a main script with the '-m' flag. The following is completely broken because the code is not yet written. 

In [2]:
!python -m doctest -v phone_how_to.rst

Trying:
    from phones import PhoneNumber
Expecting nothing
**********************************************************************
File "phone_how_to.rst", line 4, in phone_how_to.rst
Failed example:
    from phones import PhoneNumber
Exception raised:
    Traceback (most recent call last):
      File "/home/yannpaul/project/blog/blog_env/lib/python2.7/doctest.py", line 1315, in __run
        compileflags, 1) in test.globs
      File "<doctest phone_how_to.rst[0]>", line 1, in <module>
        from phones import PhoneNumber
    ImportError: No module named phones
Trying:
    PhoneNumber(1112223333, 'home')
Expecting:
    [home] (111) 222-3333
**********************************************************************
File "phone_how_to.rst", line 8, in phone_how_to.rst
Failed example:
    PhoneNumber(1112223333, 'home')
Exception raised:
    Traceback (most recent call last):
      File "/home/yannpaul/project/blog/blog_env/lib/python2.7/doctest.py", line 1315, in __run
        compileflag

Now let's write the code:

In [3]:
%%file phones.py
from __future__ import print_function


class NeedAreaCode(Exception):
    """Name says it all"""
    pass


class InvalidePhoneNumber(Exception):
    """When the number just has the wrong digits"""
    pass


class PhoneNumber(object):
    """
    A basic phone number class, with some validation for area code and length.
    
    >>> PhoneNumber(1112223333, 'home')
    [home] (111) 222-3333
    >>> PhoneNumber("1112223333", 'home')
    [home] (111) 222-3333
    >>> PhoneNumber("11112223333", 'home')
    [home] (111) 222-3333
    >>> PhoneNumber("111-222-3333", 'home')
    [home] (111) 222-3333
    >>> PhoneNumber('1-(800)-666-7777', 'office')
    [office] (800) 666-7777
    >>> PhoneNumber('1 2 3 4 5 6 7 8 9 0', 'home')
    [home] (123) 456-7890
    
    """

    def __init__(self, number, tag):
        number = self._clean_number(number)
        self._validate_number(number)
        self._number = number
        self._area = number[0:3]
        self._mid = number[3:6]
        self._last = number[6:]
        self.tag = tag

    def _clean_number(self, number):
        number = (str(number)
                  .replace(' ', '')
                  .replace('-', '')
                  .replace('(', '')
                  .replace(')', ''))
        if len(number) == 11 and number[0] == '1':
            number = number[1:]
        return number

    def _validate_number(self, number):
        ndigits = len(number)
        if ndigits == 7:
            raise NeedAreaCode('for number {}'.format(number))
        elif ndigits == 10:
            pass
        else:
            raise InvalidePhoneNumber('for number {}'.format(number))

    def __str__(self):
        return "[{}] ({}) {}-{}".format(self.tag,
                                        self._area,
                                        self._mid,
                                        self._last)

    def __repr__(self):
        return str(self)

    @property
    def raw_number(self):
        return self._number


Writing phones.py


In [4]:
!python -m doctest -v phone_how_to.rst

Trying:
    from phones import PhoneNumber
Expecting nothing
ok
Trying:
    PhoneNumber(1112223333, 'home')
Expecting:
    [home] (111) 222-3333
ok
Trying:
    PhoneNumber("1112223333", 'home')
Expecting:
    [home] (111) 222-3333
ok
Trying:
    PhoneNumber("11112223333", 'home')
Expecting:
    [home] (111) 222-3333
ok
Trying:
    number = PhoneNumber("111-222-3333", 'home')
Expecting nothing
ok
Trying:
    number
Expecting:
    [home] (111) 222-3333
ok
Trying:
    number.raw_number
Expecting:
    '1112223333'
ok
1 items passed all tests:
   7 tests in phone_how_to.rst
7 tests in 1 items.
7 passed and 0 failed.
Test passed.


A keen eye will have found tests embedded in the code's docstring. You can run these test in a similar fashion:

In [5]:
!python -m doctest -v phones.py

Trying:
    PhoneNumber(1112223333, 'home')
Expecting:
    [home] (111) 222-3333
ok
Trying:
    PhoneNumber("1112223333", 'home')
Expecting:
    [home] (111) 222-3333
ok
Trying:
    PhoneNumber("11112223333", 'home')
Expecting:
    [home] (111) 222-3333
ok
Trying:
    PhoneNumber("111-222-3333", 'home')
Expecting:
    [home] (111) 222-3333
ok
Trying:
    PhoneNumber('1-(800)-666-7777', 'office')
Expecting:
    [office] (800) 666-7777
ok
Trying:
    PhoneNumber('1 2 3 4 5 6 7 8 9 0', 'home')
Expecting:
    [home] (123) 456-7890
ok
9 items had no tests:
    phones
    phones.InvalidePhoneNumber
    phones.NeedAreaCode
    phones.PhoneNumber.__init__
    phones.PhoneNumber.__repr__
    phones.PhoneNumber.__str__
    phones.PhoneNumber._clean_number
    phones.PhoneNumber._validate_number
    phones.PhoneNumber.raw_number
1 items passed all tests:
   6 tests in phones.PhoneNumber
6 tests in 10 items.
6 passed and 0 failed.
Test passed.


The annoying thing about docstring doctests is that they each have their own context, so any 'setUp' needs to be done repeatedly for each docstring. This is not the case for the document based doctests. For further reading, there's a really good [module-of-the-week](http://pymotw.com/2/doctest/) summary of the `doctest` module.

So now that I have decent `doctest` exposure, what else would I present in a tutorial?

Well, for a list of utilities I would cover:

* [doctest](https://docs.python.org/2/library/doctest.html)
* [unittest](https://docs.python.org/2/library/unittest.html)
* [mock](https://github.com/testing-cabal/mock)
* [coverage](http://coverage.readthedocs.org/en/coverage-4.0b1/)
* [nose](https://nose.readthedocs.org/en/latest/)
* [pytest](http://pytest.org/latest/)
* [behave](http://pythonhosted.org/behave/)
* [ddt](careers.stackoverflow.com)
* [faker](http://www.joke2k.net/faker/)

For concepts, it would be just the basics (and I would probably make Ned's talk a prerequeset):

* unit testing
* regression tests
* test coverage
* mocking
* test driven development (test first)
* behavior driven development
* continuous integration

Those lists are long, so I can already see this will be a two parter :)